# Programming Assignment II: Explainability

In this assignment you will train machine learning models and experiment with techniques discussed in the lectures.
This assignment makes use of existing Python libraries for some questions. We have provided links to tutorials/examples if you're not familiar with them yet.

All code that you implement should be in this notebook. You should submit:
* This notebook with your code added. Make sure to add enough documentation. Also provide complete answers to the more theoretical questions in this notebook. These questions are followed by an 'answer indent':
> Answer:

The notebook .ipynb should have the name format `Prog_Explainability_Group_X.ipynb`, where X is your programming group ID.

Important notes:
* Deadline for this assignment is **Friday, May 30, 17:00**.
* Send it to both Maria Muratidi (m.mouratidi@uu.nl) and Heysem Kaya (h.kaya@uu.nl), CCing your programming partner.
* Title of the email: [INFOMHCML] Explainability programming assignment submission [X], with X the number of your group.
* There will be a lab session to assist you with the assignment on **Tuesday, May 27, between 11:00-12:45 at DALTON 500 - 6.27 and DALTON 500 - 8.27**.

### Installation

For this assignment, we are going to use the following Python packages:
graphviz, matplotlib, pandas, statsmodels, openpyxl, interpret, and scikit-learn.

In [1]:
# Installing packages
!pip install graphviz
!pip install matplotlib pandas statsmodels openpyxl
!pip install interpret
!pip install scikit-learn --upgrade

     ---------------------------------------- 16.6/16.6 MB 9.0 MB/s eta 0:00:00
     -------------------------------------- 544.2/544.2 kB 8.6 MB/s eta 0:00:00
     -------------------------------------- 778.9/778.9 kB 9.9 MB/s eta 0:00:00
  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)
     ---------------------------------------- 4.0/4.0 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached dash_html_components-2.0.0-py3-none-any.whl (4.1 kB)
     ---------------------------------------- 7.8/7.8 MB 6.8 MB/s eta 0:00:00
  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)
     ---------------------------------------- 1.7/1.7 MB 8.1 MB/s eta 0:00:00
     -------------------------------------- 259.8/259.8 kB 7.8 MB/s eta 0:00:00
     -------------------------------------- 295.0/295.0 kB 6.1 MB/s eta 0:00:00
     -------------------------------------- 181.3/181.3 kB 5.3 MB/s et

ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


     --------------------------------------- 11.1/11.1 MB 10.1 MB/s eta 0:00:00
     -------------------------------------- 307.7/307.7 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.1
    Uninstalling scikit-learn-1.2.1:
      Successfully uninstalled scikit-learn-1.2.1


### Read the data
We are going to use the ChaLearn LAP-FI (First Impressions) Dataset. This dataset contains 10.000 data points, which correspond to videos collected from YouTube and annotated via Amazon Mechanical Turk for the BIG-5 personality impressions: openness, extraversion, conscientiousness, neuroticism, agreeableness.

These five personality impression scores will be used as features to predict the outcome variable: a job interview invitation.

For a detailed description, see the [paper of the dataset](https://ieeexplore.ieee.org/abstract/document/7966041?casa_token=1Y03H5ykCqsAAAAA:VLhCcjAgByJ2hTdKhulmIUiXIVepEJfFyB7HM0XVts7bN8Gi8wMsiTT0qZ--I_kq8wiUHIpPN7es).


1.   If you use Google Colab, upload 'all_df.csv' (you can find this file on blackboard) through the upload button in the Files tab.
  - Copy the path of the file;
  - Run the cell below with your path. This will ask you for permission to access your Google Drive files and then you can access the data.
2.   If you are running this notebook at your own machine (jupyter notebook), locate the 'all_df.csv' file in the same folder this notebook exists. Then you can run the second cell below.

   



In [ ]:
# Run this cell only if you use Google Colab
from google.colab import drive
drive.mount('/content/gdrive')

# Make sure you uploaded all_df.csv to your Google Drive and change the path
# to the directory it is located in (usually in content/gdrive/MyDrive/...)
%cd  '/content/gdrive/MyDrive/HCML/Explainability'

In [2]:
# Run this cell (both when working locally or with Google Colab)
import pandas as pd

data = pd.read_csv("all_df.csv")
print("Data loaded")

Data loaded


### Loading and preprocessing the data
There are 6000, 2000 and 2000 examples for training, validation/development and test set respectively. In the data this is indicated by the feature `split`.

The training set is used to train models, the validation/development set to optimize the models hyper-parameters, and the test set to evaluate the trained model.

In [3]:
import numpy as np

# global variables
FEATURE_NAMES = ['extraversion', 'neuroticism', 'agreeableness', 'conscientiousness', 'openness']
LABEL_NAME = 'interview'

def load_data():
    def split_feature_label(data_set):
        features = data_set[FEATURE_NAMES]
        labels = data_set[LABEL_NAME]
        return features, labels

    train_set = data[data['split'] == 'training']
    val_set = data[data['split'] == 'validation']
    test_set = data[data['split'] == 'test']

    train_features, train_labels = split_feature_label(train_set)
    val_features, val_labels = split_feature_label(val_set)
    test_features, test_labels = split_feature_label(test_set)

    return train_features, train_labels, val_features, \
        val_labels, test_features, test_labels

# Load the data with the function above
(train_features, train_labels, dev_features, \
        dev_labels, test_features, test_labels) = load_data()

# Part 1. Different models

### **1. Linear Regression**

Train a linear regression model (we recommend the `statsmodels.api` package with the ordinary least squares model `sm`).

Hint: to get a linear regression model, you should manually add a constant variable (usually called bias or intercept - that has a fixed value of 1 for all instances) to the data, either by adding it column yourself or by using the `add_constant()` function.



**Q1.1**

Provide the $R^2$ (goodness of fit) statistic and for each feature (+ the bias variable), the following in tabular format:
* Weight estimate (coef)
* SE (standard error of estimates)
* T-statistic

Hint: You can print the summary of the model using `.summary()` to do this. This gives an extensive overview of the performance of a model.

In [5]:
# We recommend the statsmodels package
import statsmodels.api as sm

# Your code to add a bias/intercept variable
train_features_with_bias = sm.add_constant(train_features)

# Train the model and print out the summary
mod = sm.OLS(train_labels, train_features_with_bias)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:              interview   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                 1.278e+04
Date:                Sun, 25 May 2025   Prob (F-statistic):               0.00
Time:                        17:33:46   Log-Likelihood:                 10232.
No. Observations:                6000   AIC:                        -2.045e+04
Df Residuals:                    5994   BIC:                        -2.041e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.0727      0.00

**Q1.2**

Which three features are the most important?

> Answer:
- Conscientiousness
- Agreeableness
- Neuroticism
?


**Q1.3**

How does the predicted 'interview' score change with an 0.1 increase of the 'conscientiousness' feature given that all other feature values remain the same?

> Answer:
Consider that 

**Q1.4**

Show bar graph illustrations of the feature effects for the first two validation set instances.

> Answer:

In [ ]:
# Compute the Feature Effects

# Show bar graphs

**Q1.5**

Reflection: why would training a regression tree not work well for this dataset in terms of model interpretability? And under what conditions could the dataset be used with a decision tree to yield an interpretable model?

> Answer:

### **2. Explainable Boosting Model**
Train an Explainable Boosting Machine (EBM) with [InterpretML](https://interpret.ml/docs/ebm.html). EBM is a Generalized Additive Model (GAM) that is highly intelligible and explainable.

The `interpret` package provides both global and local explanation functions: `explain_global()` and `explain_local()` can be used to interpret a ML model.

**Q2.1**

Visualize/provide global (model-wise) feature importances for EBM as a table or figure.

In [ ]:
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show

# EBM Global feature importances

**Q2.2**

What are the most important two features in EBM? Are they the same as in the linear model?

> Answer:

Now, to have an idea how EBM treats the input and generates the explanation, visualize EBM local explanations on a synthetic instance generated from training set mean feature vector as input and training set mean response as output.



In [ ]:
# EBM Local explanation for training set mean vector with corresponding training set mean label
train_mean_x = pd.DataFrame(train_features.mean(axis=0)).T
train_mean_y = pd.DataFrame([train_labels.mean()])

print(train_mean_x)
print(train_mean_y)

**Q2.3**

Now, visualize local (instance-wise) feature importances for the first two instances of the development set.

In [ ]:
# EBM local explanations for the first two development set instances

**Q2.4**

Let's compare these feature importances with the feature effects in question 1.4.

* Are the feature contribution orderings the same in both models for the two instances?
> Answer:

* For the second example's explanation, why do you think the contribution of *conscientiousness* is positive, while the contribution of *agreeableness* is negative? (Hint: consider the feature values relative to the training set mean values you calculated / processed in the former subquestion.)
> Answer:

# Part 2. Model-Agnostic Methods for Interpreting/Explaining NN

### **3. Training Neural Networks**
Train a one-layer Neural Network (multi-layer perceptron (MLP) Regressor, but with one layer) with the following settings:

- Activation function: ReLU
- Size of the hidden layer: 50 neurons
- Recommended optimizer/solver: Adam

For a tutorial see [Tutorial](https://scikit-learn.org/stable/modules/neural_networks_supervised.html).

**Q3.1**

Apply the trained neural network model to the development set to find the best hyperparameters (such as learning rate). Report the Root Mean Square Error (RMSE) performance measure.

**Note.** A development set RMSE below 0.045 is reasonable, then you can apply the corresponding model on the test set in the next question.

In [ ]:
from sklearn.neural_network import MLPRegressor

# Train the MLPRegressor and show RMSE on development set

**Q3.2**

Now use the best settings to report the Root Mean Square Error (RMSE) performance measure on the test set.

It is possible to use the combination of the training and development sets to retrain the model and report the test set performance. You can also use the model that was trained on the training set only.

In [ ]:
# RMSE on test set

Now we can analyze factors that influence the predictions. Both Partial Dependence Plots (PDP) and Individual Conditional Expectation (ICE) plots can be used to visualize and analyze interaction between the target response and a set of input features of interest.

See the [Documentation](https://scikit-learn.org/stable/modules/partial_dependence.html) on how to use PDPs and ICEs.

**Q3.3**

Generate univariate and bivariate PDPs for the `conscientiousness` and `agreeableness` features with the neural network you trained above.

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

# PDPs

**Q3.4**

What do these plots show?

> Answer:

**Q3.5**

Now generate ICE plots for each feature.

In [ ]:
# ICEs

**Q3.6**

What can you conclude from ICE plots above?

> Answer:

**Q3.7**

Implement the PDF (Partial Dependence Function) for univariate analysis of the trained NN model.


In [ ]:
def PDF(X, model, feature):
  """
  Input   Dataset 'X', Model 'model', feature_name 'feature'
  Output  x_values: independent variable values
          f_values: corresponding output per x value
  """
  # Note: uncomment the lines below and complete the right hand side (where you see '..' to set them to suitable values, respective explanations are provided for each variable
  # num_samples = ..  # set the number of samples/steps to slice the range of the continuous feature, e.g., 100.
  # min_val = ..      # minimum value of the given feature
  # max_val = ..      # maximum value of the given feature
  # step_size = ..    # see the algorithm in corresponsing lecture slides to calculate the step size as a function of the above variables
  # x_values = ..     # x_values at which we will calculate the partial function of the given feature
  # f_values = ..     # the calculated partial function values corresponding to x_values

  # for k in range(num_samples - 1):
    # Change part of the data according to the formula of PDF algorithm
    # Let the model predict and calculate the f_value for this k

  # return x_values, f_values

**Q3.8**

Calculate and visualize the feature importances obtained by your PDF algorithm with a bar graph. How do we calculate the feature importance given the x_values and y_values of the PDF algorithm?

In [ ]:
# First calculate the x_values and f_values for each feature

In [ ]:
from scipy.stats import linregress

# Fit a linear model per feature, what is the output of this linear model?

**Q3.9**

What are the two most important features obtained by the PDF algorithm for the MLP model? How do these two features compare to the top two features from the Linear Model and the EBM?

> Answer:


### **4. Permutation Feature Importance**

**Q4.1**

Implement the permutation feature importance algorithm using RMSE as the error function. No existing libraries (barring the RMSE from `sklearn` and a function for random sampling / permutation) are allowed to be used, you will implement it yourself with the framework below.

In [ ]:
def PFI(X, labels, model, base_rmse):
  results = []

  for feature in X:
    # Create a copy of X_test
    # Scramble the values of the given predictor
    # Calculate the new RMSE
    # Append the increase in MSE to the list of results

  # Put the results into a pandas dataframe and rank the predictors by score

  # return results_df

**Q4.2**

Visualize the feature importances obtained by your PFI algorithm with a bar graph.

In [ ]:
# Bar graph of feature importances

**Q4.3**

What are the two most important features obtained by the permutation feature importance algorithm for the MLP model? How do these two features compare to the top two features from the Linear Model and the EBM?

> Answer: